# DEA Annual Landcover Processing - Demo Workflow

This notebook demonstrates how to use the DEA annual landcover processing scripts and visualize the results.

## Overview

The DEA (Digital Earth Australia) Annual Landcover product provides Australia-wide land cover classification at 25m resolution from 1988 to present. This workflow shows how to:

1. Run the processing scripts to generate woody/non-woody classifications
2. Load and inspect the output GeoTIFFs
3. Visualize the time series
4. Create animations and summary statistics

## Prerequisites

Before running this notebook:

```bash
# Fetch state boundaries (if not already done)
python scripts/fetch_australian_state_geojson.py

# Run DEA processing (this may take some time)
python scripts/run_dea_processing.py --state nsw --start-year 2000 --end-year 2005
```

## Data Sources

- **DEA Annual Landcover**: https://www.dea.ga.gov.au/products/dea-land-cover
- **State Boundaries**: Natural Earth via `fetch_australian_state_geojson.py`
- **Configuration**: `config.yaml`

In [ ]:
# Import required libraries
import sys
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import rasterio
from rasterio.plot import show
import yaml

# Add src to path for imports
sys.path.insert(0, str(Path.cwd().parent / 'src'))

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

## 1. Load Configuration and Check Outputs

First, let's load the configuration and check what outputs are available.

In [ ]:
# Load configuration
config_path = Path.cwd().parent / 'config.yaml'
with open(config_path, 'r') as f:
    config = yaml.safe_load(f)

dea_config = config['dea_annual_landcover']

print("DEA Configuration:")
print(f"  Product: {dea_config['product_id']}")
print(f"  Time period: {dea_config['start_year']}-{dea_config['end_year']}")
print(f"  Resolution: {dea_config['resolution']}m")
print(f"  CRS: {dea_config['crs']}")
print(f"  Output directory: {dea_config['output_dir']}")

In [ ]:
# Check available outputs
output_base = Path.cwd().parent / dea_config['output_dir']

# List NSW outputs
nsw_dir = output_base / 'nsw'
if nsw_dir.exists():
    nsw_files = sorted(nsw_dir.glob('landcover_*.tif'))
    print(f"\nNSW outputs: {len(nsw_files)} files")
    if nsw_files:
        print(f"  First: {nsw_files[0].name}")
        print(f"  Last: {nsw_files[-1].name}")
else:
    print("\nNSW outputs: Not found")
    print("Run: python scripts/run_dea_processing.py --state nsw")

# List QLD outputs
qld_dir = output_base / 'qld'
if qld_dir.exists():
    qld_files = sorted(qld_dir.glob('landcover_*.tif'))
    print(f"\nQLD outputs: {len(qld_files)} files")
    if qld_files:
        print(f"  First: {qld_files[0].name}")
        print(f"  Last: {qld_files[-1].name}")
else:
    print("\nQLD outputs: Not found")
    print("Run: python scripts/run_dea_processing.py --state qld")

## 2. Visualize Single Year

Load and display a single year of landcover data.

In [ ]:
# Select a file to visualize
if nsw_dir.exists() and len(nsw_files) > 0:
    sample_file = nsw_files[0]
    state_name = "NSW"
elif qld_dir.exists() and len(qld_files) > 0:
    sample_file = qld_files[0]
    state_name = "QLD"
else:
    print("No output files found. Please run processing first.")
    sample_file = None

if sample_file:
    print(f"Loading: {sample_file.name}")
    
    # Open and read the raster
    with rasterio.open(sample_file) as src:
        data = src.read(1)
        
        print(f"\nRaster Info:")
        print(f"  Shape: {data.shape}")
        print(f"  CRS: {src.crs}")
        print(f"  Bounds: {src.bounds}")
        print(f"  Classes: {np.unique(data)}")
        
        # Plot
        fig, ax = plt.subplots(figsize=(12, 8))
        
        # Define colors: 0=gray (other), 1=darkgreen (woody), 2=lightgreen (non-woody)
        from matplotlib.colors import ListedColormap
        colors = ['#808080', '#006400', '#90EE90']  # gray, darkgreen, lightgreen
        cmap = ListedColormap(colors)
        
        im = ax.imshow(data, cmap=cmap, vmin=0, vmax=2)
        ax.set_title(f'{state_name} Landcover - {sample_file.stem.split("_")[-1]}', fontsize=14)
        ax.axis('off')
        
        # Add colorbar
        cbar = plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
        cbar.set_ticks([0, 1, 2])
        cbar.set_ticklabels(['Other', 'Woody', 'Non-woody'])
        
        plt.tight_layout()
        plt.show()
        
        # Calculate statistics
        total_pixels = data.size
        other_pct = (data == 0).sum() / total_pixels * 100
        woody_pct = (data == 1).sum() / total_pixels * 100
        nonwoody_pct = (data == 2).sum() / total_pixels * 100
        
        print(f"\nClass Distribution:")
        print(f"  Other: {other_pct:.1f}%")
        print(f"  Woody: {woody_pct:.1f}%")
        print(f"  Non-woody: {nonwoody_pct:.1f}%")

## 3. Time Series Analysis

Calculate and visualize changes in woody/non-woody vegetation over time.

In [ ]:
# Extract time series statistics from all files
def extract_statistics(file_list, state):
    """Extract statistics from a list of GeoTIFF files."""
    stats = []
    
    for file_path in file_list:
        # Extract year from filename
        year = int(file_path.stem.split('_')[-1])
        
        # Read data
        with rasterio.open(file_path) as src:
            data = src.read(1)
            
            total = data.size
            other = (data == 0).sum()
            woody = (data == 1).sum()
            nonwoody = (data == 2).sum()
            
            stats.append({
                'year': year,
                'state': state,
                'other': other,
                'woody': woody,
                'nonwoody': nonwoody,
                'total': total,
                'other_pct': other / total * 100,
                'woody_pct': woody / total * 100,
                'nonwoody_pct': nonwoody / total * 100
            })
    
    return pd.DataFrame(stats)

# Extract statistics for available states
dfs = []

if nsw_dir.exists() and len(nsw_files) > 0:
    nsw_stats = extract_statistics(nsw_files, 'NSW')
    dfs.append(nsw_stats)
    print(f"Processed {len(nsw_files)} NSW files")

if qld_dir.exists() and len(qld_files) > 0:
    qld_stats = extract_statistics(qld_files, 'QLD')
    dfs.append(qld_stats)
    print(f"Processed {len(qld_files)} QLD files")

if dfs:
    all_stats = pd.concat(dfs, ignore_index=True)
    print(f"\nTotal records: {len(all_stats)}")
    display(all_stats.head())
else:
    print("No data to process")
    all_stats = None

In [ ]:
# Plot time series
if all_stats is not None:
    fig, axes = plt.subplots(2, 1, figsize=(14, 10))
    
    # Plot 1: Woody vegetation percentage over time
    for state in all_stats['state'].unique():
        state_data = all_stats[all_stats['state'] == state].sort_values('year')
        axes[0].plot(state_data['year'], state_data['woody_pct'], 
                    marker='o', label=state, linewidth=2)
    
    axes[0].set_xlabel('Year', fontsize=12)
    axes[0].set_ylabel('Woody Vegetation (%)', fontsize=12)
    axes[0].set_title('Woody Vegetation Cover Over Time', fontsize=14, fontweight='bold')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Plot 2: Stacked area chart
    for state in all_stats['state'].unique():
        state_data = all_stats[all_stats['state'] == state].sort_values('year')
        
        axes[1].fill_between(state_data['year'], 0, state_data['woody_pct'],
                            label=f'{state} - Woody', alpha=0.7)
        axes[1].fill_between(state_data['year'], 
                            state_data['woody_pct'], 
                            state_data['woody_pct'] + state_data['nonwoody_pct'],
                            label=f'{state} - Non-woody', alpha=0.7)
    
    axes[1].set_xlabel('Year', fontsize=12)
    axes[1].set_ylabel('Vegetation Cover (%)', fontsize=12)
    axes[1].set_title('Vegetation Cover Composition', fontsize=14, fontweight='bold')
    axes[1].legend(loc='upper left')
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

## 4. Change Detection

Identify areas where land cover has changed between two time periods.

In [ ]:
# Compare first and last years
if nsw_files and len(nsw_files) >= 2:
    first_file = nsw_files[0]
    last_file = nsw_files[-1]
    
    # Read both files
    with rasterio.open(first_file) as src:
        first_data = src.read(1)
    
    with rasterio.open(last_file) as src:
        last_data = src.read(1)
    
    # Calculate change
    # Positive = increase in class value, Negative = decrease
    change = last_data.astype(int) - first_data.astype(int)
    
    # Plot change map
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    # First year
    axes[0].imshow(first_data, cmap=cmap, vmin=0, vmax=2)
    axes[0].set_title(f'Year {first_file.stem.split("_")[-1]}', fontsize=12)
    axes[0].axis('off')
    
    # Last year
    axes[1].imshow(last_data, cmap=cmap, vmin=0, vmax=2)
    axes[1].set_title(f'Year {last_file.stem.split("_")[-1]}', fontsize=12)
    axes[1].axis('off')
    
    # Change
    change_plot = axes[2].imshow(change, cmap='RdYlGn', vmin=-2, vmax=2)
    axes[2].set_title('Change (Last - First)', fontsize=12)
    axes[2].axis('off')
    
    plt.colorbar(change_plot, ax=axes[2], fraction=0.046, pad=0.04,
                label='Change in Class')
    
    plt.tight_layout()
    plt.show()
    
    # Calculate change statistics
    print("\nChange Statistics:")
    print(f"  No change: {(change == 0).sum():,} pixels")
    print(f"  Decrease (clearing): {(change < 0).sum():,} pixels")
    print(f"  Increase (growth): {(change > 0).sum():,} pixels")
    print(f"  Net change: {change.sum():,} pixels")

## 5. Export Results

Export statistics to CSV for further analysis or visualization in other tools.

In [ ]:
# Save statistics to CSV
if all_stats is not None:
    output_csv = output_base / 'timeseries_statistics.csv'
    all_stats.to_csv(output_csv, index=False)
    print(f"Statistics saved to: {output_csv}")
    print(f"\nRows: {len(all_stats)}")
    print(f"Columns: {list(all_stats.columns)}")

## 6. View Animation

If you've created an animation using the processing script, you can display it here.

In [ ]:
# Display animation if it exists
from IPython.display import Image, display

animation_files = []
if nsw_dir.exists():
    nsw_anim = nsw_dir / 'animation.gif'
    if nsw_anim.exists():
        animation_files.append(('NSW', nsw_anim))

if qld_dir.exists():
    qld_anim = qld_dir / 'animation.gif'
    if qld_anim.exists():
        animation_files.append(('QLD', qld_anim))

if animation_files:
    for state, anim_path in animation_files:
        print(f"\n{state} Animation:")
        display(Image(filename=str(anim_path)))
else:
    print("No animation files found.")
    print("Animations are created automatically when running run_dea_processing.py")

## Summary

This notebook demonstrated:

1. ✓ Loading and inspecting DEA landcover processing outputs
2. ✓ Visualizing single-year landcover maps
3. ✓ Creating time series of vegetation cover statistics
4. ✓ Detecting land cover changes between time periods
5. ✓ Exporting results for further analysis
6. ✓ Viewing animations

## Next Steps

1. **Customize class mapping** in `config.yaml` to refine woody/non-woody definitions
2. **Process more years** by adjusting `--start-year` and `--end-year` parameters
3. **Add spatial analysis** (e.g., regional statistics, hotspot detection)
4. **Integrate with R** for publication-quality visualizations
5. **Compare with other datasets** (SLATS, Hansen Forest Change, etc.)

## Troubleshooting

**No output files found:**
```bash
# Run processing script first
python scripts/run_dea_processing.py --state nsw --start-year 2000 --end-year 2005
```

**Missing state boundaries:**
```bash
# Download boundaries
python scripts/fetch_australian_state_geojson.py
```

**For real DEA data access:**
- Install: `pip install pystac-client odc-stac datacube`
- See: https://knowledge.dea.ga.gov.au/

## References

- DEA Annual Landcover: https://www.dea.ga.gov.au/products/dea-land-cover
- Project repository: https://github.com/HMB3/AUS_Land_Clearing
- DEA Knowledge Hub: https://knowledge.dea.ga.gov.au/